In [ ]:
import pandas as pd
from data_processing import *

# DataProcessing

Dieses Jupyter Notebook dient dem Import, der Analyse, und der Aufbereitung von Daten der Individualanmeldung.
Dabei ist es als lebendes Dokument gedacht, d.h. es soll vor Beginn der Datenaufbereitung kopiert und, wo nötig, angepasst werden.

Das Notebook ist in folgende Schritte unterteilt:

1. Daten laden und teilweise umformatieren
    - Spaltennamen aus Forms sind super lang, daher in einfache Namen verwandeln
    - Clubnamen sind häufig auch etwas sperrig ('Debattierclub Aachen e.V.' -- schließlich sind wir alle Debattierclubs, und die Info, dass es ein eingetragener Verein ist für das Tabben auch nicht von herausragender Bedeutung)
2. Datenkonsistenz analysieren
    - stimmen Teamnamen?
    - ist die richtige Anzahl Jurierender / Teams / Teammitglieder vorhanden?
3. Clashes eintragen
4. Diverses zu beachten
5. Export als CSV für OpenTab

## Daten laden und umformatieren

In [ ]:
df_registration = pd.read_csv('IndReg.csv')
df_registration = df_registration.applymap(lambda x: x if type(x) != str else x.strip())

In [ ]:
# Nur diese Columns werden übernommen, der Rest wird ignoriert
columns = {
    'timestamp': 'Timestamp',
    'club': 'Für welchen Club meldest du dich an?',
    'first_name': 'Vorname',
    'last_name': 'Nachname',
    'email': 'Email-Adresse',
    'tel': 'Telefonnummer',
    'role': 'Redest oder Jurierst du?',
    'team': 'Wie lautet euer Teamname?',
    'team_members': 'Mit wem redest du im Team?',
    'show_name': 'Ich bin Teilnehmerin oder Teilnehmer auf der Campus-Debatte Göttingen 2023 und bin damit einverstanden, dass ich mit meinem vollständigen Klarnamen aufgeführt werde.',
    'judge_clashes_text': 'Gibt es Teams, Personen oder Clubs, die du nicht neutral jurieren kannst/möchtest? (Bitte niemanden aus dem eigenen Club eintragen) \nHinweis: Wenn ihr eine längere Liste potentieller Clashes habt, gibt es die Möglichkeit nach Veröffentlichung der Zimmereinteilungen in einem seperaten Formular eure dann konkreten Clashes anzugeben. ',
    'judge_tournaments': 'An wie vielen OPD-Turnieren hast du bereits als Juror:in teilgenommen?',
    'judge_experience': 'Was hast du schon für Juriererfahrungen?',
    'judge_breaks': 'Wie oft bist du schon auf OPD-Turnieren als Juror:in gebreakt?',
    'judge_chaired': 'Wie oft hast du schon auf einem OPD-Turnier hauptjuriert?',
    'judge_bias': 'Wie bepunktest du erfahrungsgemäß?',
    'daf': 'Erfüllst du die Kriterien zur Teilnahme in der Kategorie "Deutsch als Fremdsprache"?',
    'comments': 'Gibt es sonst noch etwas, das du uns mitteilen möchtest?',
    'team_clashes_text': 'Gibt es Personen, von denen du nicht neutral juriert werden kannst/möchtest? (Bitte niemanden aus dem eigenen Club eintragen)',
    'judge_nachwuchs': 'Erfüllst du die Voraussetzungen, um für die beste Nachwuchs-Jurierleistung ausgezeichnet zu werden? (Hast du auf weniger als fünf Turnieren juriert und ist deine erste Turniererfahrung als Redner:in oder Juror:in weniger als drei Jahre her?)'
}

In [ ]:
df_registration = map_column_names(df_registration, columns)
df_registration.set_index('email', inplace=True)

In [ ]:
# Convenience-Spalten
df_registration['judge'] = df_registration.role == 'Ich juriere'
df_registration['full_name'] = df_registration['first_name'] + " " + df_registration['last_name']

In [ ]:
club_name_map = {
    'Streitkultur Tübingen': 'SK',
    'der Universität Münster e.V.': 'Münster',
    'Wortgefechte Potsdam': 'Potsdam',
    'FAU Erlangen-Nürnberg': 'Nürnberg',
    'JG Mainz': 'Mainz',
    'Debattiergesellschaft Jena': 'Jena',
    'Streitkultur Berlin e. V.': 'SKBerlin',
    'Berlin Debating Union': 'BDU',
    'Mannheim Debating Union': 'Mannheim',
    'Heidelberg': 'DCH'
}

In [ ]:
# Clubnamen in ein weniger sperriges Format überführen (Nur Stadtname, außer Streitkultur (SK), Rederei, DCH)
df_registration.club = df_registration.club.apply(lambda x: x.replace('Debattierclub', '').replace('DC', '').strip()).apply(lambda x: club_name_map[x] if x in club_name_map else x)

In [ ]:
# Daten korrigieren

corrections = {
}

for email, corr_dict in corrections.items():
    for column, val in corr_dict.items():
        df_registration.loc[email,column] = val

## Datenkonsistenz

In [ ]:
# Welche Clubs sind angemeldet? (Auch prüfen: Sind die Namen einheitlich und sind alle Namen in das weniger sperrige Format überführt worden?)
df_registration.club.unique()

In [ ]:
df_registration.sort_values('club', inplace=True)

In [ ]:
# Welche Teams haben nicht genau 3 Leute, die sich angemeldet haben?
team_counts = df_registration.sort_values('club').groupby(['club', 'team']).count()['first_name']
team_counts[team_counts != 3]

In [ ]:
# Wie viele Teams sind korrekt angemeldet?
print('Correct teams: ', len(team_counts[team_counts == 3]))

In [ ]:
# Gibt es genug DaFler, um damit etwas zu machen?
df_registration[df_registration.daf == 'Ja']

In [ ]:
# Welche Clubs erfüllen aktuell nicht die Jurierendenregelung (Annahme: Regelung ist `n`)
df_reg_count = df_registration.sort_values('club').groupby(['club', 'judge']).count()['first_name'].unstack(1)
df_reg_count.columns=['Speaker', 'Judge']
df_reg_count[df_reg_count.Speaker / 3 != df_reg_count.Judge]

In [ ]:
# Wie viele Jurierende gibt es insgesamt
df_reg_count.Judge.sum()

## Clashes

In [ ]:
# Man clasht ja eig. immer mit dem Club der Anmeldung; sonst ggfs. korrigieren.
df_registration['club_clashes'] = df_registration['club']

In [ ]:
# Hier werden weitere Clubs eingetragen, gegen die Leute clashen

club_clashes = {
    ## JUDGES
    'email@domain.com': ['Aachen', 'BDU'], # Hier ggfs. Kommentar für spätere Referenz einfügen
    ## SPEAKERS
    
}

In [ ]:
# Hier werden persönliche Clashes eingetragen

speaker_clashes = {
    ## JUDGES
    'email@domain.com': ['Vorname Nachname'],
    ## SPEAKERS
}

In [ ]:
# Alle Leute mit eingetragenen clashes (Spalte ggfs. anpassen)
df_registration[df_registration.judge_clashes_text.notna()].loc[:,['club', 'first_name', 'last_name', 'judge', 'judge_clashes_text', 'team_clashes_text', 'comments', 'club_clashes']]

In [ ]:
# Eintragen der Clashes in df_registration

for email, clashes in club_clashes.items():
    df_registration.loc[email, 'club_clashes'] += "," + ",".join(clashes)
    df_registration.loc[email, 'other_clubs'] = ",".join(clashes)

for email, clashes in speaker_clashes.items():
    df_registration.loc[email, 'person_clashes'] = ",".join(clashes)

In [ ]:
# Kontrolle

df_registration[(df_registration.other_clubs.notna() | df_registration.person_clashes.notna())].loc[:, ['full_name', 'club', 'judge', 'other_clubs', 'person_clashes']].fillna("")

## Zu beachten, aber erst in OpenTab eintragbar

In [ ]:
# Möchten nur mit Initialien auf dem Tab stehen

df_registration[df_registration.show_name != 'Ja']

## Export für OpenTab

opentab import values:

```
Full Name
First Name
Last Name
Clubs
Role
Clashes
E-Mail
```

In [ ]:

df_registration.team.fillna('#00', inplace=True)

In [ ]:
# Werte auswählen
df_export = df_registration.loc[:,['team', 'first_name', 'last_name', 'full_name', 'club_clashes', 'person_clashes']]

# Jurierende bekommen als Team `#00`, die 0en bedeuten Kompetenz als Wing bzw. Chair.
# Wird erst hinterher eingetragen, muss aber für den Import stimmen.
df_export.team.fillna('#00', inplace=True)

# Jurierende können im Nachhinein nicht hinzugefügt werden, daher hier Placeholder mit Star Trek Flavor.
df_extras = pd.DataFrame([
    {'email': 'data@hoppe.io', 'team': '#40', 'first_name': 'Commander', 'last_name': 'Data', 'full_name': 'Commander Data', 'club_clashes': 'Reserve'},
    {'email': 'garak@hoppe.io', 'team': '#40', 'first_name': 'Elim', 'last_name': 'Garak', 'full_name': 'Ilem Garak', 'club_clashes': 'Reserve'},
    {'email': 'jellico@hoppe.io', 'team': '#40', 'first_name': 'Cpt. Edward', 'last_name': 'Jellico', 'full_name': 'Cpt. Edward Jellico', 'club_clashes': 'Reserve'},
    {'email': 'quark@hoppe.io', 'team': '#40', 'first_name': '', 'last_name': 'Quark', 'full_name': 'Quark', 'club_clashes': 'Reserve'}
]).set_index('email')

df_export = pd.concat((df_export, df_extras))

# Das Sortieren sortiert Jurierende korrekt ein; insbesondere landen die Placeholder für einfache Deaktivierung ganz am Ende der Liste.
df_export.sort_values('team').to_csv('opentab.csv')